In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.drawing.image import Image
import win32com.client as win32
import os
from PIL import ImageGrab
import win32com.client

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [17]:
query = """
select "sliceDate","sellerId","sellerName","customerName","materialName","netPrice","currency","demandQuantity","netValue","plannedShipmentDate","plannedInvoiceDate" 
from xxwh."sapDealsInternalMarket" 
where "sliceDate" = (select max("sliceDate") as max_data from xxwh."sapDealsInternalMarket")
"""
data=export_from_WHWEEK(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:168: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [69]:
print_to_excel = True

In [ ]:
data_work = data
# Merge data
data_work['Comp_segment'] = merge_SalesUnits(data_work,'sellerId',merge_col='ocpSegment')
data_work['Cpty_id'] = merge_Mapping(data_work,'customerName')
data_work['Cpty_segment'] = merge_SalesUnits(data_work,'Cpty_id',merge_col='ocpSegment')
data_work['Comp_holding'] = merge_SalesUnits(data_work,'sellerId',merge_col='holding')
data_work['Cpty_holding'] = merge_SalesUnits(data_work,'Cpty_id',merge_col='holding')
# Counterclaims
data_counterclaims = data_work[data_work.Cpty_id!='External']
data_counterclaims_pivot = data_counterclaims.pivot_table(index=['Comp_segment','sellerId','Cpty_segment','Cpty_id','materialName','currency'], values=['demandQuantity','netValue'], aggfunc=sum).reset_index()
data_counterclaims_pivot['Comp_Cpty'] = data_counterclaims_pivot[['sellerId','Cpty_id','materialName','currency']].astype(str).agg('.'.join, axis=1)
data_counterclaims_pivot['Cpty_Comp'] = data_counterclaims_pivot[['Cpty_id','sellerId','materialName','currency']].astype(str).agg('.'.join, axis=1)
data_merge = data_counterclaims_pivot.merge(data_counterclaims_pivot, how='left', left_on='Comp_Cpty', right_on='Cpty_Comp')
data_merge = data_merge.iloc[:,[0,1,2,3,4,5,6,7,16,17]]
data_merge['Amount_is_equal'] = data_merge.netValue_x==data_merge.netValue_y
data_counterclaims_final = data_merge.sort_values(['demandQuantity_y','sellerId_x','Cpty_id_x'])
# manual map
manual_map = data_work.loc[data_work.Comp_segment=='External',['sellerId','sellerName','Comp_segment','Comp_holding']].drop_duplicates()
# Empty data
data_empty = data_work[data_work.isna().any(axis=1)]
data_empty_sellerName = data_empty[data_empty.sellerName.isna()]
assert data_empty.shape ==data_empty_sellerName.shape
data_empty_final = data_empty_sellerName[['sellerId','sellerName']].drop_duplicates()

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SGC' 'SUEK RU' 'SUEK RU' ... 'SUEK RU' 'SUEK RU' 'SUEK RU']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', f'{id_col}_merge']
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAP

In [74]:
if print_to_excel == True:
    Output_file = f'{str(date.today())}_xxwh-sapDealsInternalMarket.xlsx'
    manual_map.to_excel(Output_file, sheet_name='Manual_map', index=False)
    new_list(data_counterclaims_final,Output_file,'Counterclaims')
    new_list(data_work,Output_file,'Data')
    new_list(data_empty_final,Output_file,'Empty_data')